In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CSE426_Data_Mining_and_Warehouse_Lab/LAB_4/employee_data.csv')
data

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
0              1    2149.014246               8.357787             82.027862   
1              2    1958.520710               8.560785             87.003125   
2              3    2194.306561               9.083051             90.026218   
3              4    2456.908957               9.053802             90.234903   
4              5    1929.753988               6.622331             87.749673   
..           ...            ...                    ...                   ...   
195          196    2115.595214               7.530824             87.449918   
196          197    1734.842769               6.286865             88.650625   
197          198     492.649900               9.353872             59.558590   
198          199    2017.462616               7.885460             87.778534   
199          200    1657.108911               9.237816             91.886502   

     Absenteeism_Days  Overtime_Hours  
0                   0       10.573297  
1                   2       10.139310  
2                   2        5.920432  
3                   0       12.238761  
4                   0       11.936453  
..                ...             ...  
195                 0       11.039465  
196                 1       12.994030  
197                13        1.311234  
198                 0       16.265124  
199                 0        9.581231  

[200 rows x 6 columns]

In [ ]:
from sklearn.preprocessing import StandardScaler

numerical_cols = ['Monthly_Sales', 'Customer_Satisfaction', 'Task_Completion_Rate', 'Absenteeism_Days', 'Overtime_Hours']

scaler = StandardScaler()

data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

data.head()

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
0            1       0.545506               0.303572             -1.047948   
1            2       0.015983               0.518344             -0.281555   
2            3       0.671407               1.070905              0.184125   
3            4       1.401374               1.039959              0.216271   
4            5      -0.063981              -1.532551             -0.166556   

   Absenteeism_Days  Overtime_Hours  
0         -0.728194        0.104498  
1          0.572153       -0.041920  
2          0.572153       -1.465285  
3         -0.728194        0.666392  
4         -0.728194        0.564400

In [ ]:
from scipy.spatial.distance import mahalanobis

# Select the features for Mahalanobis distance calculation
features = ['Monthly_Sales', 'Customer_Satisfaction', 'Task_Completion_Rate', 'Absenteeism_Days', 'Overtime_Hours']
X = data[features]

# Calculate the mean vector
mean_vector = np.mean(X, axis=0)

# Calculate the covariance matrix
cov_matrix = np.cov(X, rowvar=False)

# Calculate the inverse of the covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

In [ ]:
# Function to calculate Mahalanobis Distance

def calculate_mahalanobis(x):
    return mahalanobis(x, mean_vector, inv_cov_matrix)

In [ ]:
# Calculate Mahalanobis distances for all data points
mahalanobis_distances = X.apply(calculate_mahalanobis, axis=1)

data['Mahalanobis_Distance'] = mahalanobis_distances

outlier_threshold = 3
data['Mahalanobis_Distance_Outliers'] = (data['Mahalanobis_Distance'] > outlier_threshold)

data

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
0              1       0.545506               0.303572             -1.047948   
1              2       0.015983               0.518344             -0.281555   
2              3       0.671407               1.070905              0.184125   
3              4       1.401374               1.039959              0.216271   
4              5      -0.063981              -1.532551             -0.166556   
..           ...            ...                    ...                   ...   
195          196       0.452609              -0.571359             -0.212730   
196          197      -0.605785              -1.887475             -0.027772   
197          198      -4.058761               1.357436             -4.509134   
198          199       0.179826              -0.196153             -0.162110   
199          200      -0.821865               1.234648              0.470685   

     Absenteeism_Days  Overtime_Hours  Mahalanobis_Distance  \
0           -0.728194        0.104498              1.771345   
1            0.572153       -0.041920              0.790501   
2            0.572153       -1.465285              2.092761   
3           -0.728194        0.666392              1.866704   
4           -0.728194        0.564400              1.793438   
..                ...             ...                   ...   
195         -0.728194        0.261774              1.118547   
196         -0.078021        0.921205              2.113868   
197          7.724063       -3.020336              8.487572   
198         -0.728194        2.024806              2.226668   
199         -0.728194       -0.230205              2.192195   

     Mahalanobis_Distance_Outliers  
0                            False  
1                            False  
2                            False  
3                            False  
4                            False  
..                             ...  
195                          False  
196                          False  
197                           True  
198                          False  
199                          False  

[200 rows x 8 columns]

In [8]:
from sklearn.neighbors import LocalOutlierFactor

# Initialize the Local Outlier Factor model
lof = LocalOutlierFactor(n_neighbors=20, contamination='auto')
lof.fit(X)

# Get the outlier scores
data['LOF_Score'] = lof.negative_outlier_factor_

# Identify outliers based on a threshold (e.g., scores below -1.5)
threshold_lof = -1.5
data['LOF_Outliers'] = (data['LOF_Score'] < threshold_lof)

data

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
0              1       0.545506               0.303572             -1.047948   
1              2       0.015983               0.518344             -0.281555   
2              3       0.671407               1.070905              0.184125   
3              4       1.401374               1.039959              0.216271   
4              5      -0.063981              -1.532551             -0.166556   
..           ...            ...                    ...                   ...   
195          196       0.452609              -0.571359             -0.212730   
196          197      -0.605785              -1.887475             -0.027772   
197          198      -4.058761               1.357436             -4.509134   
198          199       0.179826              -0.196153             -0.162110   
199          200      -0.821865               1.234648              0.470685   

     Absenteeism_Days  Overtime_Hours  Mahalanobis_Distance  \
0           -0.728194        0.104498              1.771345   
1            0.572153       -0.041920              0.790501   
2            0.572153       -1.465285              2.092761   
3           -0.728194        0.666392              1.866704   
4           -0.728194        0.564400              1.793438   
..                ...             ...                   ...   
195         -0.728194        0.261774              1.118547   
196         -0.078021        0.921205              2.113868   
197          7.724063       -3.020336              8.487572   
198         -0.728194        2.024806              2.226668   
199         -0.728194       -0.230205              2.192195   

     Mahalanobis_Distance_Outliers  LOF_Score  LOF_Outliers  
0                            False  -1.028132         False  
1                            False  -0.975992         False  
2                            False  -1.148195         False  
3                            False  -1.141210         False  
4                            False  -1.021101         False  
..                             ...        ...           ...  
195                          False  -0.971931         False  
196                          False  -1.039211         False  
197                           True  -4.428641          True  
198                          False  -1.143595         False  
199                          False  -1.065941         False  

[200 rows x 10 columns]

In [9]:
data['Combined_Outliers'] = data['Mahalanobis_Distance_Outliers'] & data['LOF_Outliers']

data[data['Combined_Outliers']]

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
5              6      -4.056373              -1.067193             -4.870521   
13            14      -4.064083               0.614207             -2.738584   
20            21       1.353522               2.041042              1.720495   
40            41      -3.904363              -0.913460             -2.454236   
62            63      -0.791314              -3.504248             -0.358845   
71            72       1.413888              -1.594119             -1.592863   
125          126       1.957955              -1.359587             -1.210041   
137          138      -4.032241              -0.414997             -5.496167   
179          180       2.399696              -0.930153              1.061011   
193          194      -0.907566               1.862725              0.437075   
197          198      -4.058761               1.357436             -4.509134   

     Absenteeism_Days  Overtime_Hours  Mahalanobis_Distance  \
5            3.172847        2.100596              5.892779   
13           5.123368        0.099954              5.449213   
20          -0.078021        1.120634              3.134824   
40           3.823021        1.780134              4.878778   
62          -0.078021       -0.773524              3.662635   
71          -0.728194       -2.001400              3.840240   
125          0.572153       -1.124130              3.727420   
137          2.522674       -0.553376              6.028794   
179          2.522674        2.013039              5.724780   
193          0.572153       -2.111379              3.088399   
197          7.724063       -3.020336              8.487572   

     Mahalanobis_Distance_Outliers  LOF_Score  LOF_Outliers  Combined_Outliers  
5                             True  -3.538358          True               True  
13                            True  -3.198208          True               True  
20                            True  -1.528536          True               True  
40                            True  -2.953425          True               True  
62                            True  -1.651594          True               True  
71                            True  -1.647135          True               True  
125                           True  -1.515688          True               True  
137                           True  -3.562036          True               True  
179                           True  -2.084320          True               True  
193                           True  -1.526913          True               True  
197                           True  -4.428641          True               True

**Isolation Forest**

In [10]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(contamination='auto', random_state=42)
isolation_forest.fit(X)

# Predict outliers (-1 for outliers, 1 for inliers)
data['Isolation_Forest_Outliers'] = isolation_forest.predict(X)

# Convert predictions to boolean (1 for inliers, -1 for outliers)
data['Isolation_Forest_Outliers'] = data['Isolation_Forest_Outliers'] == -1

data[data['Isolation_Forest_Outliers']]

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
5              6      -4.056373              -1.067193             -4.870521   
9             10       0.583738               2.041042              0.576310   
13            14      -4.064083               0.614207             -2.738584   
20            21       1.353522               2.041042              1.720495   
21            22      -0.056996              -2.050546              1.532057   
34            35       0.817223               2.041042              1.337208   
36            37       0.305461              -2.217581              1.434453   
40            41      -3.904363              -0.913460             -2.454236   
62            63      -0.791314              -3.504248             -0.358845   
71            72       1.413888              -1.594119             -1.592863   
73            74       1.436077              -0.064142              1.232859   
74            75      -2.053381              -1.113410              1.447048   
78            79       0.207806              -0.001118              1.720495   
82            83       1.363734               1.603046             -1.057205   
84            85      -0.542937               2.041042             -0.402458   
125          126       1.957955              -1.359587             -1.210041   
137          138      -4.032241              -0.414997             -5.496167   
146          147      -0.900237              -1.607050              0.624550   
157          158       0.526424              -0.380371             -0.153072   
161          162       0.787652               1.546677              1.720495   
179          180       2.399696              -0.930153              1.061011   
182          183      -0.761757              -2.322062              0.667458   
193          194      -0.907566               1.862725              0.437075   
197          198      -4.058761               1.357436             -4.509134   

     Absenteeism_Days  Overtime_Hours  Mahalanobis_Distance  \
5            3.172847        2.100596              5.892779   
9            1.222326        1.507683              3.127180   
13           5.123368        0.099954              5.449213   
20          -0.078021        1.120634              3.134824   
21          -0.078021       -1.630599              3.166180   
34           0.572153        0.157840              2.771811   
36          -0.078021       -1.384768              3.126055   
40           3.823021        1.780134              4.878778   
62          -0.078021       -0.773524              3.662635   
71          -0.728194       -2.001400              3.840240   
73          -0.728194       -2.542225              3.013213   
74          -0.078021       -1.692052              3.635150   
78           1.222326       -1.067636              3.011649   
82          -0.728194        1.432729              3.075453   
84           1.222326       -0.312567              2.313497   
125          0.572153       -1.124130              3.727420   
137          2.522674       -0.553376              6.028794   
146          1.222326       -0.722078              2.635701   
157         -0.078021       -2.972044              3.089690   
161         -0.728194       -1.222183              2.586321   
179          2.522674        2.013039              5.724780   
182         -0.728194        0.953870              2.768815   
193          0.572153       -2.111379              3.088399   
197          7.724063       -3.020336              8.487572   

     Mahalanobis_Distance_Outliers  LOF_Score  LOF_Outliers  \
5                             True  -3.538358          True   
9                             True  -1.361740         False   
13                            True  -3.198208          True   
20                            True  -1.528536          True   
21                            True  -1.361707         False   
34                           False  -1.344076     

**One Class SVM**

In [11]:
from sklearn.svm import OneClassSVM

ocsvm = OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')
ocsvm.fit(X)

# Predict outliers (-1 for outliers, 1 for inliers)
data['OCSVM_Outliers'] = ocsvm.predict(X)

# Convert predictions to boolean (True for outliers)
data['OCSVM_Outliers'] = data['OCSVM_Outliers'] == -1

data[data['OCSVM_Outliers']]

Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
5              6      -4.056373              -1.067193             -4.870521   
13            14      -4.064083               0.614207             -2.738584   
20            21       1.353522               2.041042              1.720495   
27            28       0.444588              -1.215229              0.708486   
29            30      -0.111966               0.644049             -1.174502   
40            41      -3.904363              -0.913460             -2.454236   
62            63      -0.791314              -3.504248             -0.358845   
73            74       1.436077              -0.064142              1.232859   
74            75      -2.053381              -1.113410              1.447048   
78            79       0.207806              -0.001118              1.720495   
84            85      -0.542937               2.041042             -0.402458   
101          102      -0.219501              -0.667644              1.650727   
110          111      -1.468823              -0.305277              0.388394   
113          114       2.185438               0.785729             -0.518923   
123          124      -1.037751               2.041042              0.619564   
125          126       1.957955              -1.359587             -1.210041   
137          138      -4.032241              -0.414997             -5.496167   
179          180       2.399696              -0.930153              1.061011   
182          183      -0.761757              -2.322062              0.667458   
190          191      -0.241075              -1.025497              0.235207   
197          198      -4.058761               1.357436             -4.509134   

     Absenteeism_Days  Overtime_Hours  Mahalanobis_Distance  \
5            3.172847        2.100596              5.892779   
13           5.123368        0.099954              5.449213   
20          -0.078021        1.120634              3.134824   
27          -0.078021        1.815073              2.334698   
29          -0.078021       -2.314420              2.735547   
40           3.823021        1.780134              4.878778   
62          -0.078021       -0.773524              3.662635   
73          -0.728194       -2.542225              3.013213   
74          -0.078021       -1.692052              3.635150   
78           1.222326       -1.067636              3.011649   
84           1.222326       -0.312567              2.313497   
101         -0.728194        0.309342              1.977984   
110         -0.078021        1.668117              2.489737   
113         -0.078021       -0.578681              2.751498   
123         -0.078021        0.032568              2.717193   
125          0.572153       -1.124130              3.727420   
137          2.522674       -0.553376              6.028794   
179          2.522674        2.013039              5.724780   
182         -0.728194        0.953870              2.768815   
190          1.872500        0.652613              2.674596   
197          7.724063       -3.020336              8.487572   

     Mahalanobis_Distance_Outliers  LOF_Score  LOF_Outliers  \
5                             True  -3.538358          True   
13                            True  -3.198208          True   
20                            True  -1.528536          True   
27                           False  -1.141232         False   
29                           False  -1.335157         False   
40                            True  -2.953425          True   
62                            True  -1.651594          True   
73                            True  -1.454400         False   
74                            True  -1.485542         False   
78                            True  -1.291423         False   
84                           False  -1.306347         False   
101                          False  -1.070437         False   
110                          False  -1

**DBSCAN Based Outliers**

In [12]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(X)

# Get the cluster labels
data['DBSCAN_Labels'] = dbscan.labels_

# Identify outliers (points assigned to the noise cluster, labeled -1)
data['DBSCAN_Outliers'] = data['DBSCAN_Labels'] == -1

data[data['DBSCAN_Outliers']]


Employee_ID  Monthly_Sales  Customer_Satisfaction  Task_Completion_Rate  \
0              1       0.545506               0.303572             -1.047948   
1              2       0.015983               0.518344             -0.281555   
2              3       0.671407               1.070905              0.184125   
3              4       1.401374               1.039959              0.216271   
4              5      -0.063981              -1.532551             -0.166556   
..           ...            ...                    ...                   ...   
195          196       0.452609              -0.571359             -0.212730   
196          197      -0.605785              -1.887475             -0.027772   
197          198      -4.058761               1.357436             -4.509134   
198          199       0.179826              -0.196153             -0.162110   
199          200      -0.821865               1.234648              0.470685   

     Absenteeism_Days  Overtime_Hours  Mahalanobis_Distance  \
0           -0.728194        0.104498              1.771345   
1            0.572153       -0.041920              0.790501   
2            0.572153       -1.465285              2.092761   
3           -0.728194        0.666392              1.866704   
4           -0.728194        0.564400              1.793438   
..                ...             ...                   ...   
195         -0.728194        0.261774              1.118547   
196         -0.078021        0.921205              2.113868   
197          7.724063       -3.020336              8.487572   
198         -0.728194        2.024806              2.226668   
199         -0.728194       -0.230205              2.192195   

     Mahalanobis_Distance_Outliers  LOF_Score  LOF_Outliers  \
0                            False  -1.028132         False   
1                            False  -0.975992         False   
2                            False  -1.148195         False   
3                            False  -1.141210         False   
4                            False  -1.021101         False   
..                             ...        ...           ...   
195                          False  -0.971931         False   
196                          False  -1.039211         False   
197                           True  -4.428641          True   
198                          False  -1.143595         False   
199                          False  -1.065941         False   

     Combined_Outliers  Isolation_Forest_Outliers  OCSVM_Outliers  \
0                False                      False           False   
1                False                      False           False   
2                False                      False           False   
3                False                      False           False   
4                False                      False           False   
..                 ...                        ...             ...   
195              False                      False           False   
196              False                      False           False   
197               True                       True            True   
198              False                      False           False   
199              False                      False           False   

     DBSCAN_Labels  DBSCAN_Outliers  
0               -1             True  
1               -1             True  
2               -1             True  
3               -1             True  
4               -1             True  
..             ...              ...  
195             -1             True  
196             -1             True  
197             -1             True  
198             -1             True  
199             -1             True  

[200 rows x 15 columns]